In [ ]:
pip install cirq

  Using cached cirq-1.4.1-py3-none-any.whl.metadata (7.4 kB)
  Using cached cirq_aqt-1.4.1-py3-none-any.whl.metadata (1.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
import cirq
import numpy as np
import time
import threading

class QuantumChatApp:
    def __init__(self, num_qubits=10):
        self.num_qubits = num_qubits
        self.shared_key = None
        self.messages = []

    def generate_quantum_key(self):
        # Alice generates random bits and bases
        alice_bits = np.random.randint(2, size=self.num_qubits)
        alice_bases = np.random.randint(2, size=self.num_qubits)

        # Alice prepares qubits
        qubits = cirq.LineQubit.range(self.num_qubits)
        alice_circuit = cirq.Circuit()

        for i in range(self.num_qubits):
            if alice_bases[i] == 0:  # Z basis
                if alice_bits[i] == 1:
                    alice_circuit.append(cirq.X(qubits[i]))
            else:  # X basis (Hadamard)
                if alice_bits[i] == 1:
                    alice_circuit.append(cirq.X(qubits[i]))
                alice_circuit.append(cirq.H(qubits[i]))

        # Bob generates random bases and measures
        bob_bases = np.random.randint(2, size=self.num_qubits)
        bob_circuit = cirq.Circuit()

        for i in range(self.num_qubits):
            if bob_bases[i] == 1:  # Measure in X basis
                bob_circuit.append(cirq.H(qubits[i]))
            bob_circuit.append(cirq.measure(qubits[i], key=f'qubit-{i}'))

        # Run the quantum circuit
        simulator = cirq.Simulator()
        final_circuit = alice_circuit + bob_circuit
        result = simulator.run(final_circuit)

        # Extract Bob's measurement results
        bob_bits = np.array([int(result.measurements[f'qubit-{i}']) for i in range(self.num_qubits)])

        # Compare bases and keep only matching results
        matching_bases = alice_bases == bob_bases
        self.shared_key = alice_bits[matching_bases]

    def encrypt_message(self, message):
        if self.shared_key is None:
            raise ValueError("Shared key not generated yet.")

        binary_message = ''.join(format(ord(c), '08b') for c in message)
        encrypted_message = ''.join(str(int(b) ^ k) for b, k in zip(binary_message, np.tile(self.shared_key, len(binary_message)//len(self.shared_key) + 1)))
        return encrypted_message

    def decrypt_message(self, encrypted_message):
        if self.shared_key is None:
            raise ValueError("Shared key not generated yet.")

        decrypted_binary = ''.join(str(int(b) ^ k) for b, k in zip(encrypted_message, np.tile(self.shared_key, len(encrypted_message)//len(self.shared_key) + 1)))
        decrypted_message = ''.join(chr(int(decrypted_binary[i:i+8], 2)) for i in range(0, len(decrypted_binary), 8))
        return decrypted_message

    def send_message(self, sender, message):
        encrypted_message = self.encrypt_message(message)
        self.messages.append((sender, encrypted_message))

    def display_messages(self):
        for sender, encrypted_message in self.messages:
            decrypted_message = self.decrypt_message(encrypted_message)
            print(f"{sender}: {decrypted_message}")

    def run(self):
        print("Generating quantum key...")
        self.generate_quantum_key()
        print("Quantum key generated!")

        print("\nWelcome to the Quantum Chat App!")
        print("Type 'quit' to exit the chat.")

        def receive_messages():
            while True:
                time.sleep(1)
                if self.messages:
                    print("\nNew message received!")
                    self.display_messages()
                    self.messages.clear()

        # Start the message receiving thread
        threading.Thread(target=receive_messages, daemon=True).start()

        while True:
            message = input("You: ")
            if message.lower() == 'quit':
                break
            self.send_message("You", message)
            print(self.encrypt_message(message))
        print("Chat ended. Goodbye!")

if __name__ == "__main__":
    app = QuantumChatApp()
    app.run()

<ipython-input-6-913ad810eaa1>:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  bob_bits = np.array([int(result.measurements[f'qubit-{i}']) for i in range(self.num_qubits)])


Generating quantum key...
Quantum key generated!

Welcome to the Quantum Chat App!
Type 'quit' to exit the chat.
0011111010001000101101111101101100000001111111011100110000011001100111111011011111010011

New message received!
You: Hello world
001101111000110010101001110011100000111110110011100110110011011110000011101110101101100100001010

New message received!
You: Aaryan Anand
001101011000010110111010110111100001101010111100110101010000111110001100111110111110010000000111101100111101110000011110110011011001010111010010000010011011010010011011

New message received!
You: Chaitanya Singh Negi 
001001011000010110111010110001000000011010111100110101010001110110001100111110111110010000000110101110001101000000011110100011001010100110010111001111011011010111011010000001001000000010111010

New message received!
You: Shashanka Shekhar Sharma
You: quit
Chat ended. Goodbye!
